In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
import warnings
warnings.filterwarnings("ignore")
import pandas
from pathlib import Path
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# Read in test features csv

In [2]:
# import kc_house_data_test_features.csv, drop 'Unnamed' column
df_test = pd.read_csv('../csv/kc_house_data_test_features.csv', index_col = 'id').drop(columns='Unnamed: 0')
pd.set_option("display.max_columns", None)
df_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,
1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [3]:
df_test.shape

(4322, 19)

In [4]:
df_test['zipcode'].unique()

array([98034, 98029, 98077, 98059, 98115, 98019, 98155, 98125, 98112,
       98103, 98126, 98107, 98117, 98006, 98042, 98109, 98177, 98133,
       98146, 98053, 98072, 98027, 98030, 98004, 98023, 98065, 98038,
       98166, 98052, 98105, 98119, 98198, 98058, 98032, 98014, 98136,
       98118, 98075, 98092, 98010, 98055, 98070, 98199, 98074, 98102,
       98011, 98116, 98045, 98003, 98040, 98033, 98056, 98002, 98188,
       98031, 98144, 98008, 98001, 98007, 98106, 98178, 98028, 98022,
       98122, 98024, 98168, 98108, 98148, 98039, 98005], dtype=int64)

# Import pickles

In [5]:
import pickle

In [6]:
infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [7]:
print(model.intercept_)
print(len(model.coef_))

-40245329.07741234
117


In [8]:
infile = open("other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [9]:
other_info

[            sqft_living  sqft_lot  waterfront  sqft_above  sqft_basement  \
 id                                                                         
 2591820310         2070      8893           0        2070              0   
 7974200820         2900      6730           0        1830           1070   
 7701450110         3770     10893           0        3770              0   
 9522300010         4560     14608           0        4560              0   
 9510861140         2550      5376           0        2550              0   
 ...                 ...       ...         ...         ...            ...   
 627300195          3240      9960           0        2020           1220   
 8819900270         1300      4000           0        1300              0   
 3816300095         1050      9876           0        1050              0   
 122069107          1900     43186           0        1300            600   
 6703100135         1330      6768           0        1330              0   

# Feature Engineering for holdout set

In [10]:
df_test.describe()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000,4322.000000
mean,3.387552,2.322362,2183.685794,12483.624017,1.738894,0.006478,0.201296,3.248727,7.905368,1936.708237,246.977557,1985.562008,64.213559,98077.792689,47.560202,-122.209680,2032.044193,10787.661268
std,0.905875,0.781894,966.620774,36670.448359,0.586154,0.080237,0.711409,0.554717,1.166759,895.156629,408.585953,30.556960,352.311262,50.971393,0.133669,0.147595,728.033502,27418.372095
min,0.000000,0.000000,290.000000,635.000000,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.185300,-122.505000,620.000000,651.000000
25%,3.000000,1.750000,1454.250000,3800.250000,1.000000,0.000000,0.000000,3.000000,7.000000,1248.500000,0.000000,1964.000000,0.000000,98033.000000,47.487400,-122.334000,1480.000000,4000.000000
50%,3.000000,2.500000,2000.000000,6000.000000,2.000000,0.000000,0.000000,3.000000,8.000000,1695.000000,0.000000,2002.500000,0.000000,98065.000000,47.567300,-122.230500,1870.000000,6000.000000
75%,4.000000,2.500000,2730.000000,9150.750000,2.000000,0.000000,0.000000,3.000000,9.000000,2520.000000,419.250000,2009.000000,0.000000,98117.000000,47.673375,-122.115000,2460.000000,9000.000000
max,10.000000,6.500000,8020.000000,871200.000000,3.500000,1.000000,4.000000,5.000000,13.000000,8020.000000,2810.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.364000,6210.000000,858132.000000


### Date

In [11]:
# Removing the time variable from date
df_test['date'] = df_test['date'].map(lambda x: x[:8])

# Altering the cleaned date into datetime type
df_test['date'] = pd.to_datetime(df_test['date'])


### Year renovated

In [12]:
#Change values of years renovated
df_test['yr_renovated'] = np.where(df_test['yr_renovated'] == 0, df_test['yr_built'], df_test['yr_renovated'])

### Bedrooms

In [13]:
# change 0 values to 1
df_test['bedrooms'] = np.where((df_test['bedrooms'] == 0), 1, df_test['bedrooms'])

In [14]:
# create dummy columns for bedrooms
dummy_zip = pd.get_dummies(df_test.bedrooms)
df_test = pd.concat([df_test,dummy_zip], axis='columns')

In [15]:
df_test = df_test.rename(columns={1: '1_bedroom', 2: '2_bedrooms', 3: '3_bedrooms', 4: '4_bedrooms', 5: '5_bedrooms',
                                    6: '6_bedrooms', 7: '7_bedrooms', 8: '8_bedrooms', 9: '9_bedrooms', 10: '10_bedrooms',
                                   11: '11_bedrooms'})

In [16]:
# add 11_bedrooms column
df_test['11_bedrooms'] = pd.Series([0 for x in range(len(df_test.index))])

In [17]:
# assign 0 values to 11_bedroom column
df_test['11_bedrooms'] = df_test['11_bedrooms'].fillna(0)

In [18]:
#change from floats to integers
#df_test['11_bedrooms'].astype(int)

id
1974300020    0
1974300020    0
3630020380    0
1771000290    0
5126310470    0
             ..
263000018     0
6600060120    0
1523300141    0
291310100     0
1523300157    0
Name: 11_bedrooms, Length: 4322, dtype: int32

In [20]:
df_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,1_bedroom,2_bedrooms,3_bedrooms,4_bedrooms,5_bedrooms,6_bedrooms,7_bedrooms,8_bedrooms,9_bedrooms,10_bedrooms,11_bedrooms
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0
1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0
3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,2005,98029,47.5472,-121.998,1470,1576,0,0,1,0,0,0,0,0,0,0,0.0
1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,1976,98077,47.7427,-122.071,1160,10565,0,0,1,0,0,0,0,0,0,0,0.0
5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,2005,98059,47.4863,-122.140,2830,7916,0,0,0,1,0,0,0,0,0,0,0.0


### Bathrooms

In [21]:
# change 0 values to 1
df_test['bathrooms'] = np.where((df_test['bathrooms'] == 0), 1, df_test['bathrooms'])

In [22]:
df_test['bathrooms'].describe()

count    4322.000000
mean        2.322594
std         0.781355
min         0.750000
25%         1.750000
50%         2.500000
75%         2.500000
max         6.500000
Name: bathrooms, dtype: float64

In [23]:
df_test['1_bathroom'] = np.where((df_test['bathrooms'] <= 1) & (df_test['bathrooms']>0), 1, 0)
df_test['2_bathrooms'] = np.where((df_test['bathrooms'] <= 2) & (df_test['bathrooms']>1), 1, 0)
df_test['3_bathrooms'] = np.where((df_test['bathrooms'] <= 3) & (df_test['bathrooms']>2), 1, 0)
df_test['4_bathrooms'] = np.where((df_test['bathrooms'] <= 4) & (df_test['bathrooms']>3), 1, 0)
df_test['5_bathrooms'] = np.where((df_test['bathrooms'] <= 5) & (df_test['bathrooms']>4), 1, 0)
df_test['6_bathrooms'] = np.where((df_test['bathrooms'] <= 6) & (df_test['bathrooms']>5), 1, 0)
df_test['7_bathrooms'] = np.where((df_test['bathrooms'] <= 7) & (df_test['bathrooms']>6), 1, 0)
df_test['8_bathrooms'] = np.where((df_test['bathrooms'] <= 8) & (df_test['bathrooms']>7), 1, 0)

### Floors

In [24]:
# create dummy columns for floors
dummy_zip = pd.get_dummies(df_test.floors)
df_test = pd.concat([df_test,dummy_zip], axis='columns')

In [25]:
df_test = df_test.rename(columns={1.0: '1_floor', 1.5: '1.5_floors', 2.0: '2_floors', 2.5: '2.5_floors', 
                                    3.0: '3_floors', 3.5: '3.5_floors'})

### Zipcode

In [26]:
# create dummy columns for zipcode
dummy_zip = pd.get_dummies(df_test.zipcode)
df_test = pd.concat([df_test,dummy_zip], axis='columns')

### Basement

In [27]:
# add a new feature for whether homes have a baesment
df_test['has_basement'] = [0 if x == 0 else 1 for x in df_test.sqft_basement]
df_test

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,1_bedroom,2_bedrooms,3_bedrooms,4_bedrooms,5_bedrooms,6_bedrooms,7_bedrooms,8_bedrooms,9_bedrooms,10_bedrooms,11_bedrooms,1_bathroom,2_bathrooms,3_bathrooms,4_bathrooms,5_bathrooms,6_bathrooms,7_bathrooms,8_bathrooms,1_floor,1.5_floors,2_floors,2.5_floors,3_floors,3.5_floors,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,has_basement
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,2005,98029,47.5472,-121.998,1470,1576,0,0,1,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,1976,98077,47.7427,-122.071,1160,10565,0,0,1,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,2005,98059,47.4863,-122.140,2830,7916,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,2014-05-21,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,2009,98103,47.6993,-122.346,1530,1509,0,0,1,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6600060120,2015-02-23,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,2014,98146,47.5107,-122.362,1830,7200,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1523300141,2014-06-23,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,2009,98144,47.5944,-122.299,1020,2007,0,1,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Bedroom to bathroom ratio

In [28]:
# add new featre--bedroom bathroom ratio
df_test['bed_bath_ratio'] = round(df_test['bedrooms']/df_test['bathrooms'], 2)
df_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,1_bedroom,2_bedrooms,3_bedrooms,4_bedrooms,5_bedrooms,6_bedrooms,7_bedrooms,8_bedrooms,9_bedrooms,10_bedrooms,11_bedrooms,1_bathroom,2_bathrooms,3_bathrooms,4_bathrooms,5_bathrooms,6_bathrooms,7_bathrooms,8_bathrooms,1_floor,1.5_floors,2_floors,2.5_floors,3_floors,3.5_floors,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,has_basement,bed_bath_ratio
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60
1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60
3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,2005,98029,47.5472,-121.998,1470,1576,0,0,1,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.20
1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,1976,98077,47.7427,-122.071,1160,10565,0,0,1,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.71
5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,2005,98059,47.4863,-122.140,2830,7916,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.45


### Yard space

In [29]:
# create new feature - yard space
df_test['yard_space'] = round(df_test.sqft_lot - (df_test.sqft_living / df_test.floors), 2)
df_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,1_bedroom,2_bedrooms,3_bedrooms,4_bedrooms,5_bedrooms,6_bedrooms,7_bedrooms,8_bedrooms,9_bedrooms,10_bedrooms,11_bedrooms,1_bathroom,2_bathrooms,3_bathrooms,4_bathrooms,5_bathrooms,6_bathrooms,7_bathrooms,8_bathrooms,1_floor,1.5_floors,2_floors,2.5_floors,3_floors,3.5_floors,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,has_basement,bed_bath_ratio,yard_space
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60,9230.0
1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60,9230.0
3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,2005,98029,47.5472,-121.998,1470,1576,0,0,1,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.20,1044.0
1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,1976,98077,47.7427,-122.071,1160,10565,0,0,1,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.71,14920.0
5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,2005,98059,47.4863,-122.140,2830,7916,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.45,6711.0


### Grade

In [30]:
# dummy grade by category
df_test['low_grade'] = np.where(df_test['grade'] < 6, 1, 0)
df_test['below_average_grade'] = np.where(df_test['grade'] == 6, 1, 0)
df_test['average_grade'] = np.where(df_test['grade'] ==7, 1, 0)
df_test['above_average_grade'] = np.where(df_test['grade'].isin(range(8,9)), 1, 0)
df_test['high_Grade'] = np.where(df_test['grade'] > 9, 1, 0)

### View

In [31]:
# create dummy columns for bedrooms
dummy_zip = pd.get_dummies(df_test.view)
df_test = pd.concat([df_test,dummy_zip], axis='columns')

In [32]:
df_test = df_test.rename(columns={0: '0_view', 1: '1_view', 2: '2_view', 3: '3_view', 
                                    4: '4_view'})

### Condition

In [33]:
# create dummy columns for bedrooms
dummy_zip = pd.get_dummies(df_test.condition)
df_test = pd.concat([df_test,dummy_zip], axis='columns')

In [34]:
df_test = df_test.rename(columns={1: '1_condition', 2: '2_condition', 3: '3_condition', 4: '4_condition', 
                                    5: '5_condition'})

In [35]:
df_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,1_bedroom,2_bedrooms,3_bedrooms,4_bedrooms,5_bedrooms,6_bedrooms,7_bedrooms,8_bedrooms,9_bedrooms,10_bedrooms,11_bedrooms,1_bathroom,2_bathrooms,3_bathrooms,4_bathrooms,5_bathrooms,6_bathrooms,7_bathrooms,8_bathrooms,1_floor,1.5_floors,2_floors,2.5_floors,3_floors,3.5_floors,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,has_basement,bed_bath_ratio,yard_space,low_grade,below_average_grade,average_grade,above_average_grade,high_Grade,0_view,1_view,2_view,3_view,4_view,1_condition,2_condition,3_condition,4_condition,5_condition
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60,9230.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60,9230.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,2005,98029,47.5472,-121.998,1470,1576,0,0,1,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.20,1044.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,1976,98077,47.7427,-122.071,1160,10565,0,0,1,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.71,14920.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,2005,98059,47.4863,-122.140,2830,7916,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.45,6711.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0


In [36]:
df_test.shape

(4322, 132)

In [37]:
df_test.columns.to_list()

['date',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 '1_bedroom',
 '2_bedrooms',
 '3_bedrooms',
 '4_bedrooms',
 '5_bedrooms',
 '6_bedrooms',
 '7_bedrooms',
 '8_bedrooms',
 '9_bedrooms',
 '10_bedrooms',
 '11_bedrooms',
 '1_bathroom',
 '2_bathrooms',
 '3_bathrooms',
 '4_bathrooms',
 '5_bathrooms',
 '6_bathrooms',
 '7_bathrooms',
 '8_bathrooms',
 '1_floor',
 '1.5_floors',
 '2_floors',
 '2.5_floors',
 '3_floors',
 '3.5_floors',
 98001,
 98002,
 98003,
 98004,
 98005,
 98006,
 98007,
 98008,
 98010,
 98011,
 98014,
 98019,
 98022,
 98023,
 98024,
 98027,
 98028,
 98029,
 98030,
 98031,
 98032,
 98033,
 98034,
 98038,
 98039,
 98040,
 98042,
 98045,
 98052,
 98053,
 98055,
 98056,
 98058,
 98059,
 98065,
 98070,
 98072,
 98074,
 98075,
 98077,
 98092,
 98102,
 98103,
 98105,
 98106,
 98107,
 98108,

In [39]:
# Drop columns that have dummy columns
df_test_features = df_test.drop(['date', 'bedrooms', 'bathrooms', 'zipcode', 'grade', 'floors', 
                           'view', 'condition', '4_bedrooms', 98058, '4_bathrooms', '2_floors', 'average_grade', 
                             '2_view', '4_condition'], axis=1)

In [40]:
df_test_features.shape

(4322, 117)

In [41]:
df_test_features.columns.to_list()

['sqft_living',
 'sqft_lot',
 'waterfront',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 '1_bedroom',
 '2_bedrooms',
 '3_bedrooms',
 '5_bedrooms',
 '6_bedrooms',
 '7_bedrooms',
 '8_bedrooms',
 '9_bedrooms',
 '10_bedrooms',
 '11_bedrooms',
 '1_bathroom',
 '2_bathrooms',
 '3_bathrooms',
 '5_bathrooms',
 '6_bathrooms',
 '7_bathrooms',
 '8_bathrooms',
 '1_floor',
 '1.5_floors',
 '2.5_floors',
 '3_floors',
 '3.5_floors',
 98001,
 98002,
 98003,
 98004,
 98005,
 98006,
 98007,
 98008,
 98010,
 98011,
 98014,
 98019,
 98022,
 98023,
 98024,
 98027,
 98028,
 98029,
 98030,
 98031,
 98032,
 98033,
 98034,
 98038,
 98039,
 98040,
 98042,
 98045,
 98052,
 98053,
 98055,
 98056,
 98059,
 98065,
 98070,
 98072,
 98074,
 98075,
 98077,
 98092,
 98102,
 98103,
 98105,
 98106,
 98107,
 98108,
 98109,
 98112,
 98115,
 98116,
 98117,
 98118,
 98119,
 98122,
 98125,
 98126,
 98133,
 98136,
 98144,
 98146,
 98148,
 98155,
 98166,
 98168,

# Predict the holdout set

In [42]:
#df[['bedrooms', 'bathrooms', 'sqft_living']]

In [43]:
final_columns = list(other_info)

In [44]:
final_columns

[            sqft_living  sqft_lot  waterfront  sqft_above  sqft_basement  \
 id                                                                         
 2591820310         2070      8893           0        2070              0   
 7974200820         2900      6730           0        1830           1070   
 7701450110         3770     10893           0        3770              0   
 9522300010         4560     14608           0        4560              0   
 9510861140         2550      5376           0        2550              0   
 ...                 ...       ...         ...         ...            ...   
 627300195          3240      9960           0        2020           1220   
 8819900270         1300      4000           0        1300              0   
 3816300095         1050      9876           0        1050              0   
 122069107          1900     43186           0        1300            600   
 6703100135         1330      6768           0        1330              0   

In [45]:
#df_test_features = df_test.drop(['date', 'bedrooms', 'bathrooms', 'zipcode', 'grade', 'floors', 
#                           'view', 'condition', '4_bedrooms', '4_bathrooms', '2_floors', 'average_grade', '2_view', '4_condition', ], axis=1)

In [46]:
df_test_features.head()

,sqft_living,sqft_lot,waterfront,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,1_bedroom,2_bedrooms,3_bedrooms,5_bedrooms,6_bedrooms,7_bedrooms,8_bedrooms,9_bedrooms,10_bedrooms,11_bedrooms,1_bathroom,2_bathrooms,3_bathrooms,5_bathrooms,6_bathrooms,7_bathrooms,8_bathrooms,1_floor,1.5_floors,2.5_floors,3_floors,3.5_floors,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,has_basement,bed_bath_ratio,yard_space,low_grade,below_average_grade,above_average_grade,high_Grade,0_view,1_view,3_view,4_view,1_condition,2_condition,3_condition,5_condition
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1974300020,2270,11500,0,1540,730,1967,1967,47.7089,-122.241,2020,10918,0,0,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60,9230.0,0,0,1,0,1,0,0,0,0,0,1,0
1974300020,2270,11500,0,1540,730,1967,1967,47.7089,-122.241,2020,10918,0,0,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.60,9230.0,0,0,1,0,1,0,0,0,0,0,1,0
3630020380,1470,1779,0,1160,310,2005,2005,47.5472,-121.998,1470,1576,0,0,1,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.20,1044.0,0,0,1,0,1,0,0,0,0,0,1,0
1771000290,1280,16200,0,1030,250,1976,1976,47.7427,-122.071,1160,10565,0,0,1,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.71,14920.0,0,0,1,0,1,0,0,0,0,0,1,0
5126310470,2830,8126,0,2830,0,2005,2005,47.4863,-122.140,2830,7916,0,0,0,0,0,0,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.45,6711.0,0,0,1,0,1,0,0,0,0,0,1,0


In [47]:
df_test_features.shape

(4322, 117)

In [50]:
final_answers = model.predict(df_test_features)

In [51]:
final_answers

array([505677.08155997, 505677.08155997, 363917.11244927, ...,
       364873.53739066, 372484.34120489, 364504.67513233])

### Convert array to pandas dataframe

In [54]:
df_final = pd.DataFrame(final_answers)

In [55]:
df_final

,0
0,505677.081560
1,505677.081560
2,363917.112449
3,244489.346230
4,550734.634574
...,...
4317,526580.340199
4318,439213.659693
4319,364873.537391
4320,372484.341205


### Save final dataframe as a csv

In [59]:
df_final.to_csv('..csv/housing_preds_adam_cumurcu.csv')